In [2]:
# Import necessary packages

from models.model2 import BicycleModel
from models.model4 import FourWheelModel
from models.utils import fit_circle
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sc
from models.utils import get_csv_row_count
from models.utils import get_folder_path

NameError: name 'Dynamic_Bicycle_Linear' is not defined

In [5]:
# Generate data for Dynamic_Bicycle_Linear

np.random.seed(2023)

def get_circle_radius(model):
    
    t0 = 0
    N = model.open_loop_tf/model.dt
    
    steering = 0.2
    torque = 40
    
    inputs = [steering, torque, torque, torque, torque]


    t,x_vect = model.do_open_loop_sim_cst_inputs(t0, inputs)
    
    x = x_vect[int(N/2):,0]
    y = x_vect[int(N/2):,1]

    radius = fit_circle(x, y)
    #plt.plot(x,y)
    #plt.axis('equal')
    return radius

dt = 0.005
open_loop_tf = 50

T_peak_mean = 0.37
T_slope = 0.4




def plot_histogram_1dnparray(data):
    # Plot the histogram
    plt.hist(data, bins=30, edgecolor='black')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram of Data')
    plt.grid(True)

    # Show the plot
    plt.show()

data_size = 1
T_peak_vector = np.full(data_size, T_peak_mean)
radiuses = np.empty(data_size)
for i in range(data_size):
    T_peak_vector[i] += 0 #np.random.normal(0,0.1)

plot_histogram_1dnparray(T_peak_vector)

velocity = 5
#Generate circles
for i in range(data_size):
    model2_lin = Dynamic_Bicycle_Linear(velocity, dt, open_loop_tf, T_peak_vector[i], T_slope)
    radius2_lin = get_circle_radius(model2_lin)
    radiuses[i] = radius2_lin
    
    if i%10==0:
        print('Iteration ', i)

plt.plot(T_peak_vector, radiuses, '.')

# Combine x and y arrays horizontally
data = np.column_stack((T_peak_vector, radiuses))

# Save the data to a CSV file
np.savetxt('./Data/Data_Bicycle_linear/radiuses.csv', data, delimiter=',', header='T_peak,R', comments='')




Iteration  0
Iteration  10
Iteration  20
Iteration  30
Iteration  40


(0.36846083387278333, 0.010747673584785025)

In [ ]:
# Generate data for BicycleModel

np.random.seed(2023)

def get_circle_radius(model):
    
    t0 = 0
    N = model.open_loop_tf/model.dt
    
    steering = 0.2
    torque = 1000
    
    inputs = [steering, torque, torque, torque, torque]


    t,x_vect = model.do_open_loop_sim_cst_inputs(t0, inputs)
    
    x = x_vect[int(N/2):,0]
    y = x_vect[int(N/2):,1]

    radius = fit_circle(x, y)
    #plt.plot(x,y)
    #plt.axis('equal')
    return radius

dt = 0.05
open_loop_tf = 50

T_peak_mean = 0.37
T_slope_mean = 0.4




def plot_histogram_1dnparray(data):
    # Plot the histogram
    plt.hist(data, bins=30, edgecolor='black')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram of Data')
    plt.grid(True)

    # Show the plot
    plt.show()

data_size = 50
T_peak_vector = np.full(data_size, T_peak_mean)
T_slope_vector = np.full(data_size, T_slope_mean)
radiuses = np.empty(data_size)
for i in range(data_size):
    T_peak_vector[i] += np.random.normal(0,0.01)
    T_slope_vector[i] += 0

plot_histogram_1dnparray(T_peak_vector)
plot_histogram_1dnparray(T_slope_vector)

#Generate circles
for i in range(data_size):
    model2_lin = BicycleModel(dt, open_loop_tf, T_peak_vector[i], T_slope)

    radius2_lin = get_circle_radius(model2_lin)
    radiuses[i] = radius2_lin
    
    if i%10==0:
        print('Iteration ', i)

plt.plot(T_peak_vector, radiuses, '.')

# Combine x and y arrays horizontally
data = np.column_stack((T_peak_vector, T_slope_vector,radiuses))

# Save the data to a CSV file
np.savetxt('./Data/Data_Bicycle/radiuses.csv', data, delimiter=',', header='T_peak, T_slope, R', comments='')

sc.norm.fit(T_peak_vector)


In [5]:
np.random.seed(1000)
def get_open_loop_data(model, file_path):
    
    t,x_vect = model.do_open_loop_sim_from_csv(file_path)
    
    x = x_vect[:,0]
    y = x_vect[:,1]

    plt.plot(x,y, color = 'black')
    plt.xlabel(r'$x \rm [m]$')
    plt.ylabel(r'$y \rm [m]$')
    plt.axis()
    
    
    return t,x_vect

dt = 0.025 # = 1/40 = 1/control_frequency
file_path = get_folder_path()+'/open_loop_inputs/open_loop_commands.csv'
print(file_path)
N = get_csv_row_count(file_path)

open_loop_tf =N*dt

def plot_histogram_1dnparray(data):
    # Plot the histogram
    plt.hist(data, bins=30, edgecolor='black')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram of Data')
    plt.grid(True)

    # Show the plot
    plt.show()

data_size = 30
T_peak_mean = 0.37
T_slope_mean = 0.4
T_peak_vector = np.full(data_size, T_peak_mean)
T_slope_vector = np.full(data_size, T_slope_mean)
radiuses = np.empty([data_size, 1])
for i in range(data_size):
    T_peak_vector[i] += np.random.normal(0,0.01)
    T_slope_vector[i] += np.random.normal(0,0.01)

final_array = np.empty(964).reshape(-1,1)

model = FourWheelModel(dt, open_loop_tf, T_peak_vector[0], T_slope_vector[0])
t,final_array = get_open_loop_data(model, file_path)
for i in range(1, data_size):
    model = FourWheelModel(dt, open_loop_tf, T_peak_vector[i], T_slope_vector[i])
    t,x_vect = get_open_loop_data(model, file_path)
    final_array = np.hstack((final_array, x_vect))

#plt.plot(T_peak_vector, radiuses, '.')

# Combine x and y arrays horizontally
data = np.column_stack((T_peak_vector, T_slope_vector,radiuses))

# Save the data to a CSV file
np.savetxt('./Data/Data_four_wheel/trajectories.csv', final_array, delimiter=',', comments='')

np.savetxt('./Data/Data_four_wheel/priors.csv', data, delimiter=',', comments='')

sc.norm.fit(T_peak_vector)


/Users/jonas/Desktop/intergation_model/open_loop_inputs/open_loop_commands.csv


In [57]:
np.savetxt('./Data/Data_four_wheel/trajectories.csv', final_array, delimiter=',', comments='')

In [54]:
int(final_array.shape[1]/6)

2

In [55]:
final_array.shape

(964, 12)